<center><img src='https://akm-img-a-in.tosshub.com/indiatoday/images/story/202004/Screenshot_2020-04-06_at_3.36.-770x433.png?7sMd_I7U5lOVvbjtmGmSJjU3nrUvnHc5' size ='1000px'></center>

As we all know how this virus is spreading very dangerously in the communities and localities. While kaggle community is working hard to show how is the spreading trend is in the different countries of the world .The trends of COVID in india is not lesser known as it is spreading very rapidly in the indian subcontinent but measures taken by Indian government are good and essential but still we should prepare for a long fight against this pandemic until scientists all around the world could find a breakthrough.Until than STAY INDOORS and follow the guidelinces of the authorities. <font color="red" size=3>STAY SAFE</font>

<font color="red" size=3>Please UPVOTE this kernel if you like it. It motivates me to produce more quality content :) <br><br>
    Please do comment what your views are and what you understand from this.
    <br><br> Dont't forget to give your suggestions in the comment section </font>

This notebook is my contribution to show how the medical situations are in India and what are estimates and trends.<br>
Credit - Whole Kaggle community but special thanks to<br>
1. [Logistic Curve Fitting - Global Covid-19 Confirmed](https://www.kaggle.com/dferhadi/logistic-curve-fitting-global-covid-19-confirmed) ~ by Daner Ferhadi for curve fitting 

<font color="red" size=3>It might take some time to load </font>

# Contents

* [<font size=4>Findings</font>](#1)
    * [Number of Patients](#1.1)
    * [Daily increase in Patients](#1.2)
    * [Explaining the trends](#1.3)
    * [Deaths and Mortality](#1.4)
    * [Age Group](#1.5)
    * [Sample tested](#1.6)
    * [Hospital and Beds](#1.7)
    * [Correlation](#1.8)
    * [curve fit](#1.9)
  
* [<font size=4>My Thoughts</font>](#2)

In [ ]:
!pip install chart_studio
!pip install pgeocode

In [ ]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"
from plotly.subplots import make_subplots

from pathlib import Path
data_dir = Path('../input/covid19-corona-virus-india-dataset')

import os
os.listdir(data_dir)

In [ ]:
df = pd.read_csv('../input/covid19-corona-virus-india-dataset/patients_data.csv')

In [ ]:
from IPython.display import display, HTML
display(HTML(df.tail().to_html()))

Shows the information about the number of patients this dataset is updated in every 24 hours till today we see their are 11512 cases of covid in india<br>
> a little info - i started making this notebook yesterday and there where about near 9500 patients so you can see how sevierly the number is increasing

In [ ]:
def state_wise_patients(name,df=df):
    data = df.loc[df['detected_state']==name]
    df = data[['patient_number','date_announced','detected_state']]
    data = df.groupby('date_announced')['patient_number'].nunique()
    data = data.reset_index()
    data['date_announced']=pd.to_datetime(data['date_announced'],format = '%d/%m/%Y')
    data = data.sort_values(by=['date_announced'], ascending=True)
    data['patient_number'] = data.patient_number.cumsum()
    return data

This function segregates and collects the data of different states with dates and patient_number

In [ ]:
collection = {}
for i in df.detected_state.unique():
    collection['patients in '+ str(i)] = state_wise_patients(i)

> for instance 
see collection['patients in Maharashtra']

In [ ]:
collection['patients in Maharashtra']

In [ ]:
keys = list(collection.keys())

In [ ]:
visible_True=[]
for i in range(len(keys)):
    visible_True.append(True)
def t2f(i):
    visible = []
    for a in range(len(keys)):
        if a == i:
            visible.append(True)
        else:
            visible.append(False)
    return visible

In [ ]:
def create_buttons(keys=keys):
    l=[dict(label = 'All',
                  method = 'update',
                  args = [{'visible': visible_True},
                          {'title': 'All',
                           'showlegend':True}])]
    for i in range(len(keys)):
        l.append(dict(label = keys[i],
                  method = 'update',
                  args = [{'visible': t2f(i)}, # the index of True aligns with the indices of plot traces
                          {'title': keys[i],
                           'showlegend':True}]))
    return l

In [ ]:
fig = go.Figure()
keys = list(collection.keys())
for column in collection:
    fig.add_trace(
        go.Line(
            x = collection[column].date_announced,
            y = collection[column].patient_number,
            name = column
        )
    )
    
#fig.update_layout(updatemenus=[go.layout.Updatemenu( active=0,buttons=list(create_buttons()))])

fig.show()

we will analyze these states
1. Maharashtra
2. Uttar pradesh
3. Rajasthan
4. Tamil Nadu
5. Delhi 
<br>
<br>
because of these states in total comprises of 60% of india's covid +ve cases

### 1. Findings - Number of Patients <a id="1.1"></a>
you can see the plots of these states either by clicking the legend or by the dropdown list
1. As you have seen in maharasshtra the situation started getting bad in the period of 29 - 3 april and then their is near exponancial growth shows that their might had been an outbreak or a system failure
2. Though the condition is uttar pradesh was not that harsh as it was in maharashtra but the conditions goes out of handle in the period of 29 - 5 april when cluster spreading starts
3. In rajasthan Same period of  29 - 5 april is time when it starts to grow in an exponancial manner
4. Okay see again the time bound of 29 - 5 april here we se a sudden growth 30 March(67) - 5 april(571) 9x time growth
5. Delhi is too different as the period was 1-5 april when the outbreak happens

In [ ]:
def per_day_inc(collection=collection):
    for i in list(collection.keys()):
        collection[i]['prev_patients'] = collection[i]['patient_number'].shift(1)
        collection[i]['new_patients'] = collection[i]['patient_number'] - collection[i]['prev_patients']
    return collection
coll1 = per_day_inc()

In [ ]:
fig = go.Figure()
keys = list(collection.keys())
for column in collection:
    fig.add_trace(
        go.Line(
            x = collection[column].date_announced,
            y = collection[column].new_patients,
            name = column
        )
    )
    
fig.update_layout(updatemenus=[go.layout.Updatemenu(active=0,buttons=list(create_buttons()))])

fig.show()

### 2. Findings - Daily increase in  Patients <a id="1.2"></a>
you can see the plots of these states either by clicking the legend or by the dropdown list
1. In Maharastra the daily increase in number of patients were 117 till 8 April but the sudden increase is noted on april 9 and april 13 
2. Thought uttar pradesh saw a spike on april 4 and after 11 we see peaks on 13 april(75) and 14 april(102)
3. Well in rajasthan we see spike on 5 april but latter we saw spikes on 9 and after this day we saw sudden growth of 317 in number of patients 
4. Tamil Nadu see unprecedental increase on 1 April around 100 patients +ve in a single day explains why 29 - 5 April period in tamil nadu was severe
5. Though Delhi has seen up and down in the daily rate of +ve cases but 13 april saw 356 patients in a day 

In [ ]:
fig = go.Figure()
keys = list(collection.keys())
for column in collection:
    fig.add_trace(
        go.Line(
            x = collection[column].date_announced,
            y = collection[column].patient_number,
            name = column
        )
    )
    
fig.update_layout(yaxis_type='log')

fig.show()

### 3. Findings - Explaining the trends <a id="1.3"></a>
1. In log scale we see a linear growth in number of patients from 15 of March
2. Same linear growth is observed in Uttar pradesh from 8 March
3. Rajathan growth pattern matches the Uttar pradesh but rate is more
4. 18 March is the day when patient number starts grow in an exponancial manner
5. Delhi is also follows the same trend describing that Covid increases Exponancially

In [ ]:
import requests
india_data_json = requests.get('https://api.rootnet.in/covid19-in/unofficial/covid19india.org/statewise').json()
df_india = pd.io.json.json_normalize(india_data_json['data']['statewise'])
df_india = df_india.set_index("state")

In [ ]:
total = df_india.sum()
total.name = "Total"
df_t = pd.DataFrame(total,dtype=float).transpose()
df_t["Mortality Rate (per 100)"] = np.round(100*df_t["deaths"]/df_t["confirmed"],2)

In [ ]:
df_india["Mortality Rate (per 100)"]= np.round(np.nan_to_num(100*df_india["deaths"]/df_india["confirmed"]),2)
df_india.style.background_gradient(cmap='Blues',subset=["confirmed"])\
                        .background_gradient(cmap='Reds',subset=["deaths"])\
                        .background_gradient(cmap='Greens',subset=["recovered"])\
                        .background_gradient(cmap='Purples',subset=["active"])\
                        .background_gradient(cmap='plasma',subset=["Mortality Rate (per 100)"])

the above table is self explainatory it shows the most essential stats about the COVID situation (confirmed 	recovered 	deaths 	active 	Mortality Rate (per 100)) with darker color means high values

In [ ]:
trace1 = go.Bar(
                x = df_india.index,
                y = df_india.deaths,
                name = "deaths",
                marker = dict(color = 'rgba(255, 174, 255, 0.5)',
                             line=dict(color='rgb(0,0,0)',width=1.5)),
                text = df_india.index)
# create trace2 
trace2 = go.Bar(
                x = df_india.index,
                y = df_india.recovered,
                name = "recovered",
                marker = dict(color = 'rgba(255, 255, 128, 0.5)',
                              line=dict(color='rgb(0,0,0)',width=1.5)),
                text = df_india.index)
trace3 = go.Bar(
                x = df_india.index,
                y = df_india.active,
                name = "active",
                marker = dict(color = 'rgba(0, 174, 174, 0.5)',
                             line=dict(color='rgb(0,0,0)',width=1.5)),
                text = df_india.index)
data = [trace1, trace2,trace3]
layout = go.Layout(barmode = "group")
fig = go.Figure(data = data, layout = layout)
fig.update_layout(yaxis_type="log")
fig.show()

Simple bar chart in (log scale)showing the huge difference in the active and recovered cases

In [ ]:
import plotly.express as px
import numpy as np
fig = px.scatter_3d(df_india, x='recovered', y='active', z='confirmed',size='confirmed',  color=df_india.index)
fig.update_layout(height=800, width=800,scene_zaxis_type="log",scene_yaxis_type="log",scene_xaxis_type="log")
fig.show()

3D scatter plot showing a great pattern all of the data points are aligned in a linear fashion reason ??(tell in comments)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

labels = df_india.index

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels, values=df_india.deaths, name="deaths"),
              1, 1)
fig.add_trace(go.Pie(labels=labels, values=df_india["Mortality Rate (per 100)"], name="Mortality Rate (per 100)"),
              1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent+name")

fig.update_layout(
    title_text="Covid-19 ",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='deaths', x=0.18, y=0.5, font_size=20, showarrow=False),
                 dict(text='Mortality', x=0.82, y=0.5, font_size=20, showarrow=False)])
fig.show()

### 4. Findings - Deaths and Mortality <a id="1.4"></a>
1. India saw around 38.7% deaths in maharashtra and 22% in Gujarat
2. West Bengal ,Maharashtra ,Punjab, Madhya Pradesh mortality rate is significantly high as compared to other states (the mortality rate is per 100 patients) ignoring Meghalaya because of low data available 

In [ ]:
data_dir = Path('/kaggle/input/hospitalbedloc')

In [ ]:
import folium
india = folium.Map(location=[20.5937,78.9629], zoom_start=5.4)
df = pd.read_csv('../input/covid19-corona-virus-india-dataset/complete.csv')
for lat, lon,State,Death,Total_confirmed_cases in zip(df['Latitude'], df['Longitude'],df['Name of State / UT'],df['Death'],df['Total Confirmed cases']):
    folium.CircleMarker([lat, lon],
                        radius=5,
                        color='Blue',
                      popup =('State:' + str(State) + '<br>'
                             'Total Confirmed cases:' + str(Total_confirmed_cases) + '<br>',
                              'Deaths :' + str(Death) +'<br>'
                             ),
                        fill_color='red',
                        fill_opacity=0.7 ).add_to(india)
india

**geo spacial veiw of patients in different states of india (hover to view)**

Now we will analyze the Hospitals and beds available in different states

![](https://akm-img-a-in.tosshub.com/indiatoday/images/story/202003/coronaviruspatients-770x433.jpeg?FySKakF9X_ltswD07Oz2lJWHl0iRoyof)

In [ ]:
df = pd.read_csv(data_dir/'HospitalBedsIndiaLocations.csv')
df = df.fillna(0)
df['total_Beds'] = df['NumUrbanBeds_NHP18'] + df['NumRuralBeds_NHP18'] + df['NumPublicBeds_HMIS'] 
df['total_Hospitals'] = df['NumUrbanHospitals_NHP18'] + df['NumRuralHospitals_NHP18'] + df['NumSubDistrictHospitals_HMIS'] + df['NumDistrictHospitals_HMIS']

In [ ]:
#india = folium.Map(location=[20.5937,78.9629], zoom_start=5.4,max_zoom=10)
for i in range(len(df['State/UT'].values)):
    folium.Circle(
        location=[df.loc[i]['Latitude'], df.iloc[i]['Longitude']],
        tooltip = "<h5 style='text-align:center;font-weight: bold'>"+ str(df.iloc[i]['State/UT'])+"</h5>"+
        "<div style='text-align:center;'>"+"total_Beds:   " + str(df.iloc[i]['total_Beds'])+"</div>"+
        "<div style='text-align:center;'>"+"total_Hospital:   " + str(df.iloc[i]['total_Hospitals'])+"</div>"+
        "<hr style='margin:10px;'>"+
        "<ul style='color: #444;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
        "<li>NumSubDistrictHospitals_HMIS: "+str(df.iloc[i]['NumSubDistrictHospitals_HMIS'])+"</li>"+
        "<li>NumDistrictHospitals_HMIS:   "+str(df.iloc[i]['NumDistrictHospitals_HMIS'])+"</li>"+
        "<li>TotalPublicHealthFacilities_HMIS:   "+str(df.iloc[i]['TotalPublicHealthFacilities_HMIS'])+"</li>"+       
        "<li>NumPublicBeds_HMIS:   "+str(df.iloc[i]['NumPublicBeds_HMIS'])+"</li>"+
        "<li>NumRuralHospitals_NHP18:   "+str(df.iloc[i]['NumRuralHospitals_NHP18'])+"</li>"+
        "<li>NumRuralBeds_NHP18:   "+str(df.iloc[i]['NumRuralBeds_NHP18'])+"</li>"+
        "<li>NumUrbanHospitals_NHP18:   " + str(df.iloc[i]['NumUrbanHospitals_NHP18'])+"</li>"+
        "<li>NumUrbanBeds_NHP18:   " + str(df.iloc[i]['NumUrbanBeds_NHP18'])+"</li>"+
        "<li>NumCommunityHealthCenters_HMIS:   " + str(df.iloc[i]['NumCommunityHealthCenters_HMIS'])+"</li>"+
        "</ul>",
        radius=int((np.log2(df.iloc[i]['total_Beds']+1))*6000),
        color=df['State/UT'].values[i],
        fill_color='red',
        fill=True).add_to(india)
india

**Geo spacial view about the number of medical care facilities available in indian states (hover to view)**

In [ ]:
data_dir1 = Path('/kaggle/input/covid19-in-india')

In [ ]:
df1 = pd.read_csv(data_dir1/'AgeGroupDetails.csv')

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

labels = df1.AgeGroup
values = df1.TotalCases
# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'xy'}, {'type':'domain'}]])
fig.add_trace(go.Bar(x=labels, y=values, name="bar",marker = dict(color = 'rgba(0, 174, 174, 0.5)',
                             line=dict(color='rgb(0,0,0)',width=1.5)),
                text = labels),
              1, 1)
fig.add_trace(go.Pie(labels=labels, values=values, name="pie"),
              1, 2)


fig.update_layout(
    title_text="Covid-19 Age group details ")
    # Add annotations in the center of the donut pies.
fig.show()

### 5. Findings Age Group <a id="1.5"></a>
1. As you can see the patients are most in the region of 20-29,30-39 age group
2. 20-29 region comprises of 24.9% of the people that are infected while 30-39 capture 21.1% 
<br>shows that people in the these 2 regions are more infected but nothing much can be said as age group data is less avialable

Using pgeocode library to get latitude and longitude from the pincode

**Here i have tried to show the distribtution of ICMR labs around the states**
![](https://socialimages.sakalmediagroup.com/sakaltimes-prod/s3fs-public/news-story/cover-images/1ICMR_0.jpg?unHhcHI1moxtdh3EaxAFeBKxpEh1mnJE)

In [ ]:
import pgeocode
df1 =  pd.read_csv('../input/covid19-in-india/ICMRTestingLabs.csv')
nomi = pgeocode.Nominatim('IN')
lat = []
long = []
for i in df1.pincode.values:
    lat.append(nomi.query_postal_code(str(i)).latitude)
    long.append(nomi.query_postal_code(str(i)).longitude)

In [ ]:
df1['lat'] = pd.Series(lat)
df1['long'] = pd.Series(long)
df1 = df1.dropna()
df1 = df1.reset_index()

In [ ]:
import folium
#india = folium.Map(location=[20.5937,78.9629], zoom_start=5.4,max_zoom=10)
for i in range(len(df1)):
    folium.Circle(
        location=[df1.loc[i]['lat'], df1.iloc[i]['long']],
        tooltip = "<h5 style='text-align:center;font-weight: bold'>"+ str(df1.iloc[i]['state'])+"</h5>"+
        "<div style='text-align:center;'>"+"type:   " + str(df1.iloc[i]['type'])+"</div>"+
        "<div style='text-align:center;'>"+"city:   " + str(df1.iloc[i]['city'])+"</div>"+
        "<hr style='margin:10px;'>"+
        "<ul style='color: #444;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
        "<li>lab  : " + str(df1.iloc[i]['lab'])+"</li>"+
        "<li>address : " + str(df1.iloc[i]['address'])+"</li>"+
        "</ul>",
        radius=30000,
        color=df1['state'].values[i],
        fill_color='red',
        fill=True).add_to(india)
india

**This Map plot shows where are the labs are situated in different regions of india and which state has more labs(Hover to see)**

**Now i have described how the trends of testing and +ve cases connects with each other**
### data is till 27th of April 

In [ ]:
df1 = pd.read_csv('../input/covid19-in-india/ICMRTestingDetails.csv')

In [ ]:
import plotly.graph_objects as go
x1 = df1.DateTime.values
x=df1.TotalSamplesTested.values
y = df1.TotalIndividualsTested.values
z = df1.TotalPositiveCases

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x1, y=z,
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='rgb(256, 0, 256)'),
    stackgroup='one',
    name = 'TotalPositiveCases'
))
fig.add_trace(go.Scatter(
    x=x1, y=x,
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='rgb(0, 256, 256)'),
    stackgroup='one',
    name ='TotalSamplesTested'
))

fig.update_layout()
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x1, y=z,
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='rgb(256, 0, 256)'),
    stackgroup='one',
    name = 'TotalPositiveCases'
))
fig.add_trace(go.Scatter(
    x=x1, y=x,
    hoverinfo='x+y',
    mode='lines',
    line=dict(width=0.5, color='rgb(0, 256, 256)'),
    stackgroup='one',
    name ='TotalSamplesTested'
))

fig.update_layout(yaxis_type="log")
fig.show()

### 6. Findings Sample tested <a id="1.6"></a>
1. As you can see in the 1st plot the number of total samples tested are increased in a exponancial ways started(from 29 - March)
2. Though the testing rate is increasing but still number of tests are very low in comparison to the 10% population of india
3. In the second plot you can witness how is the increase in TotalSample leads to increase in Total+ve cases in a same manner leading us to infer that more and more testing should be done

In [ ]:
df = pd.read_csv(data_dir/'HospitalBedsIndiaLocations.csv')
df =df.fillna(0)
df['total_Beds'] = df['NumUrbanBeds_NHP18'] + df['NumRuralBeds_NHP18'] + df['NumPublicBeds_HMIS'] 
df['total_Hospitals'] = df['NumUrbanHospitals_NHP18'] + df['NumRuralHospitals_NHP18'] + df['NumSubDistrictHospitals_HMIS'] + df['NumDistrictHospitals_HMIS']
df.index=df['State/UT']
df = df.drop(columns=['Sno','State/UT'])

Aggregating the Beds and Hospitals in different states

In [ ]:
df1 = pd.read_csv('../input/covid19-in-india/population_india_census2011.csv')
df1 = df1.sort_values(by='State / Union Territory')
df1 = df1.reset_index()
df1.index = df1['State / Union Territory']
df1 = df1.drop(columns=['index','Sno','State / Union Territory'])
from IPython.display import display, HTML
display(HTML(df1.to_html()))

In [ ]:
df.index.values[0] = df1.index.values[0]
df.index.values[-5] = df1.index.values[-5]
df.index.values[14] = df1.index.values[13]

In [ ]:
df['Population'] = df1['Population']
df['Rural population'] = df1['Rural population']
df['Urban population'] = df1['Urban population']
df = df.drop(['Dadra & Nagar Haveli','Daman & Diu'],axis=0)

In [ ]:
df['total_Rural_Hospitals'] = df['NumRuralHospitals_NHP18'] + df['NumSubDistrictHospitals_HMIS'] + df['NumDistrictHospitals_HMIS'] 
df['total_Rural_Beds'] = df['NumRuralBeds_NHP18'] + df['NumPublicBeds_HMIS'] 
df['total_Urban_Hospitals'] =  df['NumUrbanHospitals_NHP18'] + df['NumSubDistrictHospitals_HMIS'] + df['NumDistrictHospitals_HMIS']
df['total_Urban_Beds'] = df['NumUrbanBeds_NHP18'] + df['NumPublicBeds_HMIS'] 
df['total_medical_centres'] = df['NumPrimaryHealthCenters_HMIS'] + df['NumCommunityHealthCenters_HMIS'] + df['TotalPublicHealthFacilities_HMIS']

New columns
1. total_Rural_Hospitals - total rural Hospital in the state
2. total_Rural_Beds - total rural beds in the state
3. total_Urban_Hospitals - total Urban Hospitals in the state
4. total_Urban_Beds - total Urban Beds in the state

In [ ]:
df["Hospitals (per 100000)"]= np.round(100000*df["total_Hospitals"]/df["Population"],2)
df["Beds (per 100000)"]= np.round(100000*df["total_Beds"]/df["Population"],2)
df["rural Hospitals (per 100000)"]= np.round(100000*df["total_Rural_Hospitals"]/df["Rural population"],2)
df["rural Beds (per 100000)"]= np.round(100000*df["total_Rural_Beds"]/df["Rural population"],2)
df["Urban Hospitals (per 100000)"]= np.round(100000*df["total_Urban_Hospitals"]/df["Urban population"],2)
df["Urban Beds (per 100000)"]= np.round(100000*df["total_Urban_Beds"]/df["Urban population"],2)

In [ ]:
df = df[['total_Rural_Beds','total_Urban_Hospitals','total_Urban_Beds','total_medical_centres','Hospitals (per 100000)','Beds (per 100000)'
    ,'rural Hospitals (per 100000)','rural Beds (per 100000)','Urban Hospitals (per 100000)','Urban Beds (per 100000)']]

In [ ]:
df_india = df_india.sort_index()

In [ ]:
df['confirmed'] = df_india['confirmed']
df['recovered'] = df_india['recovered']
df['deaths'] = df_india['deaths']
df['active'] = df_india['active']
df['Mortality Rate (per 100)'] = df_india['Mortality Rate (per 100)']

In [ ]:
df=df.dropna()

In [ ]:
df.style.background_gradient(cmap='Blues',subset=["Beds (per 100000)"])\
                        .background_gradient(cmap='Reds',subset=["Urban Hospitals (per 100000)"])\
                        .background_gradient(cmap='Greens',subset=["rural Hospitals (per 100000)"])\
                        .background_gradient(cmap='Purples',subset=["rural Beds (per 100000)"])\
                        .background_gradient(cmap='YlOrBr',subset=["Urban Beds (per 100000)"])\
                        .background_gradient(cmap='Oranges',subset=["Hospitals (per 100000)"])\
                        .background_gradient(cmap='Reds',subset=["Mortality Rate (per 100)"])\
                        .background_gradient(cmap='Purples',subset=["confirmed"])\
                        .background_gradient(cmap='Greens',subset=["deaths"])\
                        .background_gradient(cmap='Blues',subset=["active"])\
                        .background_gradient(cmap='Oranges',subset=["recovered"])\

### 7. Findings Hospitals and Beds <a id="1.7"></a>
1. States like Andaman and Nicobar Islands ,Sikkim ,Tripura,Himachal Pradesh has a very good number in Urban_beds per 100000 as compared to other states but keeping in mind these states has low population as compared to Uttar pradesh, Maharashtra etc.
2. Chandigrah has around 13000 rural beds for every 100000 which very good as compared to other region,Delhi also has around 5000 rural beds per 100000 which is also very good this might be a reason that delhi has such low mortality rate
3. Rajasthan , Tamil nadu ,Uttar pradesh has very low beds per person but still they have very low mortality rate which is good leading us to think that the government of these states has done some good work on social distancing
4. Madhya pradesh and Maharashtra has high mortality rate also they have low bed and hospitals per 100000 in all aspects<br>
more results can be inferred so brainstorm on it

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
                   z=df.corr(),
                   x=df.columns.values,
                   y=df.columns.values,
                   hoverongaps = False))
fig.show()

### 8. Findings Correlation <a id="1.8"></a>
1. Hospitals(per 100000)  have a correlation of -.35 with confirmed cases and -.27 with deaths
2. Urban_beds(per 100000) have a correlation values of -.33 with active cases
3. Beds(per 100000) has a -ve correlation value with Mortality Rate

In [ ]:
df = pd.read_csv('../input/covid19-corona-virus-india-dataset/patients_data.csv')
a = df.detected_district.value_counts()
a = a.dropna()

>But why does Mumbai is worst hit<br>
Maharashtra is bearing the brunt of India’s COVID-19 crisis, with 23% of the total cases and 46% of overall deaths*. Most cases are from Mumbai, with the highest share in its G-south ward. Wards with dense populations have the highest number of cases. The State has the highest number of such wards.
for more you can [see](https://www.livemint.com/news/india/coronavirus-update-mumbai-s-covid-19-case-count-breaches-10-000-mark-11588780219101.html)

![](https://media.cntraveller.in/wp-content/uploads/2020/04/BMC-mumbai-coronavirus-containment-zone-map-screen.jpg)

In [ ]:
!pip install opencage

In [ ]:
from opencage.geocoder import OpenCageGeocode
key = '157336e14b654bceb51b5fc3cc07bec4'  # get api key from:  https://opencagedata.com
geocoder = OpenCageGeocode(key)
query = 'Bid, India'  
results = geocoder.geocode(query)
lat = results[0]['geometry']['lat']
lng = results[0]['geometry']['lng']
print (lat, lng)

In [ ]:
from scipy.optimize import curve_fit
import matplotlib.pyplot as pt

In [ ]:
def log_curve(x, k, x_0, ymax):
    return ymax / (1 + np.exp(-k*(x-x_0)))
def fit_curve():
    for i in list(collection.keys()):
        if collection[i]['patient_number'].max()>650:
            x_data = range(len(collection[i].index))
            y_data = collection[i]['patient_number']
            popt, pcov = curve_fit(log_curve, x_data, y_data, bounds=([0,0,0],np.inf), maxfev=50000)
            estimated_k, estimated_x_0, ymax= popt
            k = estimated_k
            x_0 = estimated_x_0
            y_fitted = log_curve(range(0,160), k, x_0, ymax)
            fig = go.Figure()
            fig.add_trace(go.Line(x = np.arange(160),y = y_fitted,name ='predicted ' + i  ))
            fig.add_trace(go.Line(x = np.arange(80),y = y_data,name = i))
            fig.show()
fit_curve()

### 9. Findings curve fit <a id="1.9"></a>
1. Until in Delhi today we see 1561 patients and the patient count might saturate in an around more 23 days
2. For Tamil Nadu the curve shows the saturation way to early in just next 8 days.
3. In rajasthan things will take around 24 more days  to touch the peak but this might lead to around 3000+ve patients
4. In Uttar pradesh the peak will touch in more 23 days leading to 1000 +ve patients shows that uttar pradesh might have done some progress in flattening the curve
5. Same situation is in Madhya pradesh as in UP they might have taken some measures to flatten the curve peak will touch in 20 more days
6. Well I am pretty curious about the situation in Maharashtra the peak will touch in more 26 days but the rate at which the patients number increase will be very heigh until peak we will see around 8500 patients.

## New Updates
1. Zone Division <br>
starting May 4, the Union Health Ministry has issued a revised list of districts under red, orange and green zones Friday. A total of 130 districts across the country have been placed under the red zone, while 284 and 319 districts have been identified as orange and green zones, respectively.
This revised classification is based on the incidence of cases, doubling rate, the extent of testing and surveillance feedback. As per the revised criteria, Union Health Secretary Preeti Sudan, in a letter to state chief secretaries, said, green zones are districts which haven’t reported a fresh case in 21 days, down from the 28 days earlier. The orange zones are those with a few cases, and the red ones have a large number of cases.
for more you can [see](https://www.hindustantimes.com/india-news/india-s-lockdown-3-0-restrictions-red-zone-and-travel-decoded-in-9-points/story-sOpDy1zkSdL3SetlJpOaQJ.html)

![](https://pbs.twimg.com/media/EVzBrIIUYAEsdj2.jpg)

### My Thoughts <a id="2"></a>
1. I think that increasing lockdown is a good thing to be done as we might have seen in the above cell's output it is important to flatten the curve an the best thing is social distancing
2. we have seen above that the their is -ve correlation between the beds and confirmed cases so i think more medical facilities might can do the job
3. Also we should increase the testing should be increased as testing and +ve cases have linear correlation
4. Government should increase the testing facilities for better testing 
<br>Please provide your thoughts too in the comment sections


![](https://www.businessinsider.in/thumb.cms?msid=74896617&width=1200&height=900)

Well in all of this negativity we can see some remarkable changes in our environment and air quality<br>
but due to the severe fatalities covid has inflicted on makes me sad irrespective of what is the air quality during the lockdown period.<br>
but i also think that the government has spent a lot of money on to reduce the air,water pollution but we never saw this level of changes<br>
so in my though government should learn from this lockdown and try to regulate their environment saving policies